This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [1]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = '2EGJwnx8GbwgxkXqw2ru'

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import collections
import quandl
import requests
quandl.ApiConfig.api_key = "2EGJwnx8GbwgxkXqw2ru"


In [3]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
r_test = requests.get("https://www.quandl.com/api/v3/datasets/FSE/AFX_X.json?start_date=2017-01-01&end_date=2017-01-02&api_key=2EGJwnx8GbwgxkXqw2ru")
r_test.json()

{'dataset': {'id': 10095370,
  'dataset_code': 'AFX_X',
  'database_code': 'FSE',
  'name': 'Carl Zeiss Meditec (AFX_X)',
  'description': 'Stock Prices for Carl Zeiss Meditec (2019-12-27) from the Frankfurt Stock Exchange.<br><br>Trading System: Xetra<br><br>ISIN: DE0005313704',
  'refreshed_at': '2020-01-10T22:19:01.561Z',
  'newest_available_date': '2020-01-10',
  'oldest_available_date': '2000-06-07',
  'column_names': ['Date',
   'Open',
   'High',
   'Low',
   'Close',
   'Change',
   'Traded Volume',
   'Turnover',
   'Last Price of the Day',
   'Daily Traded Units',
   'Daily Turnover'],
  'frequency': 'daily',
  'type': 'Time Series',
  'premium': False,
  'limit': None,
  'transform': None,
  'column_index': None,
  'start_date': '2017-01-01',
  'end_date': '2017-01-02',
  'data': [['2017-01-02',
    34.99,
    35.94,
    34.99,
    35.8,
    None,
    44700.0,
    1590561.0,
    None,
    None,
    None]],
  'collapse': None,
  'order': None,
  'database_id': 6129}}

In [4]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
r = requests.get("https://www.quandl.com/api/v3/datasets/FSE/AFX_X.json?start_date=2017-01-01&end_date=2017-12-31&api_key=2EGJwnx8GbwgxkXqw2ru")
type(r.json())

dict

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [5]:
data = r.json()["dataset"]
data = [data["column_names"], data["data"]]
print("column names:", data[0])

column names: ['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover']


In [6]:
#For #3, we tranpose the array so the columns are now rows and its much easier to find respective min and max
data_transposed = [list(l) for l in zip(*data[1])]
#opening prices corresponds to index 1 in our data
opening_prices = list(filter(None.__ne__, data_transposed[1]))
min_open = min(opening_prices)
max_open = max(opening_prices)
print("lowest opening price:", min_open)
print("highest opening price:", max_open)

lowest opening price: 34.0
highest opening price: 53.11


In [7]:
#4
high, low = data_transposed[2], data_transposed[3]
change = [abs(h_i - l_i) for h_i, l_i in zip(high, low) if h_i and l_i]
largest_change = max(change)
print("largest change in any one day:", largest_change)

largest change in any one day: 2.8100000000000023


In [8]:
#5
#  [1,2,3,4,5]
#  [2,3,4,5]
closing_prices = data_transposed[4]
closing_copy = closing_prices[:]
closing_copy.pop(0)
closing_copy.append(closing_copy[len(closing_copy)-1])
change = [abs(c_1 - c_2) for c_1,c_2 in zip(closing_prices, closing_copy) if c_1 and c_2]
largest_change = max(change)
print("largest change in any two days:", largest_change)

largest change in any two days: 2.559999999999995


In [9]:
#6
trading_volumes = list(filter(None.__ne__, data_transposed[6]))
avg_trading_vol = sum(trading_volumes) / len(trading_volumes)
print("average trading volume:", avg_trading_vol)

average trading volume: 89124.33725490196
